In [1]:
import pandas as pd
import numpy as np
import random

# Parameters
regions = {
    "Gomti Nagar": {"grid_size": (34, 32), "active_restaurants": 153, "active_customers": 373, "depot": (17, 13)},
    "Alambagh": {"grid_size": (34, 33), "active_restaurants": 94, "active_customers": 405, "depot": (13, 18)},
    "Indira Nagar": {"grid_size": (27, 37), "active_restaurants": 87, "active_customers": 239, "depot": (21, 19)},
}

# Generate random restaurant and customer locations
def generate_locations(grid_size, num_locations):
    locations = set()
    while len(locations) < num_locations:
        x = random.randint(0, grid_size[0] - 1)
        y = random.randint(0, grid_size[1] - 1)
        locations.add((x, y))
    return list(locations)

# Calculate Manhattan distance
def manhattan_distance(cell1, cell2):
    return abs(cell1[0] - cell2[0]) + abs(cell1[1] - cell2[1])

# Generate synthetic dataset with previous order time
def generate_dataset(region_name, region_data, num_orders=1000):
    grid_size = region_data["grid_size"]
    depot = region_data["depot"]
    restaurants = generate_locations(grid_size, region_data["active_restaurants"])
    customers = generate_locations(grid_size, region_data["active_customers"])
    
    dataset = []
    courier_location = depot  # Courier starts at the depot
    previous_delivery_time = 0  # Initial previous delivery time is 0
    episode_length = 100  # Episode ends after 100 orders

    for order_id in range(1, num_orders + 1):
        restaurant = random.choice(restaurants)
        customer = random.choice(customers)
        preparation_time = random.randint(5, 15)

        # Calculate distances
        distance_to_restaurant = manhattan_distance(courier_location, restaurant)
        distance_to_customer = manhattan_distance(restaurant, customer)

        # Calculate expected delivery time using the formula
        expected_delivery_time = distance_to_customer + max(
            preparation_time, previous_delivery_time + distance_to_restaurant
        )

        # Define current state
        current_state = {
            "Restaurant Cell": restaurant,
            "Customer Cell": customer,
            "Preparation Time (mins)": preparation_time,
            "Expected Delivery Time (mins)": expected_delivery_time,
            "Distance to Depot": manhattan_distance(courier_location, depot),
            "Distance to Restaurants": [manhattan_distance(courier_location, e) for e in restaurants],
        }

        # Courier action: Accept, Reject, Return to Depot, or Move to Restaurant
        if expected_delivery_time <= 45:
            action = "Accept"
            reward = 45 - expected_delivery_time
            courier_location = customer  # Courier moves to customer after delivery
            previous_delivery_time = distance_to_customer
        else:
            action = "Reject"
            reward = -15
            previous_delivery_time = 0

        # Idle courier actions (if no order is assigned)
        if action == "Reject":
            # Randomly choose between returning to depot or moving to a restaurant
            idle_action = random.choice(["Return to Depot", "Move to Restaurant"])
            if idle_action == "Return to Depot":
                distance_to_depot = manhattan_distance(courier_location, depot)
                reward = -distance_to_depot / 10
                courier_location = depot
                previous_delivery_time = 0
            else:
                chosen_restaurant = random.choice(restaurants)
                distance_to_restaurant = manhattan_distance(courier_location, chosen_restaurant)
                reward = -distance_to_restaurant / 10
                courier_location = chosen_restaurant

        # Define next state
        next_state = {
            "Restaurant Cell": restaurant,
            "Customer Cell": customer,
            "Preparation Time (mins)": preparation_time,
            "Expected Delivery Time (mins)": expected_delivery_time,
            "Distance to Depot": manhattan_distance(courier_location, depot),
            "Distance to Restaurants": [manhattan_distance(courier_location, e) for e in restaurants],
        }

        # Done flag (episode ends after 100 orders)
        done = (order_id % episode_length == 0)

        dataset.append({
            "Region": region_name,
            "Order ID": order_id,
            "Current State": current_state,
            "Action": action,
            "Reward": reward,
            "Next State": next_state,
            "Done": done,
        })

    return dataset

# Generate dataset for all regions
all_data = []
for region_name, region_data in regions.items():
    region_dataset = generate_dataset(region_name, region_data, num_orders=1000)
    all_data.extend(region_dataset)

# Convert to DataFrame
df = pd.DataFrame(all_data)

# Save to CSV
df.to_csv("synthetic_meal_delivery_dataset_with_previous_time.csv", index=False)

print("Dataset generated and saved to 'synthetic_meal_delivery_dataset_with_previous_time.csv'.")

Dataset generated and saved to 'synthetic_meal_delivery_dataset_with_previous_time.csv'.
